<a href="https://colab.research.google.com/github/IISC-GROUP-5/Jupyter_Notebooks/blob/Pratima/Inference_file_for_Gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
import json


import pickle
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def load_model(model_file):
    """
    Load the trained model from a .pkl file.
    """
    with open(model_file, 'rb') as file:
        model = pickle.load(file)
    print(f"Model loaded successfully from {model_file}")
    return model

In [ ]:
def preprocessing(new_data):
  """
  Loads a saved model and makes predictions on new data.

  Args:
    model_path: Path to the saved model file.
    new_data: A pandas DataFrame containing new data.

  Returns:
    A list of predicted labels.
  """
  # Replace spaces in column names with underscores
  new_data.columns = new_data.columns.str.replace(' ', '_')


  # Handle missing values
  for col in new_data.columns:
    if new_data[col].isnull().sum() > 0:
      if pd.api.types.is_numeric_dtype(new_data[col]):
        # For numeric columns, fill missing values with median
        new_data[col].fillna(new_data[col].median(), inplace=True)
      else:
        # For categorical columns, fill missing values with the most frequent value
        new_data[col].fillna(new_data[col].mode()[0], inplace=True)

  # Round 'Area Income' to 2 decimal places
  new_data['Area_Income'] = new_data['Area_Income'].round(2)

  # Preprocess new data (adjust as needed)
  label_encoding_mapping = {'Female': 0, 'Male': 1}
  new_data['Gender'] = new_data['Gender'].map(label_encoding_mapping)

  return new_data


In [ ]:
def make_prediction(model, clean_data):
  """
  Loads a saved model and makes predictions on new data.

  Args:
    model_path: Path to the saved model file.
    new_data: A pandas DataFrame containing new data.

  Returns:
    A list of predicted labels.
  """
 # Ensure only the required columns are used
  clean_data = clean_data[feature_columns]

  # Make predictions
  prediction = model.predict(clean_data)
  probabilities = model.predict_proba(clean_data)
  print('Gender:' , prediction)
  print('Probabilities:', probabilities)
  return prediction, probabilities

In [ ]:
def save_inference_results(output_file, clean_data, predictions, probabilities, rawdata):
    """
    Save the inference results to a file in JSON format.
    The `raw data` is used to store input in the original form.
    """
    results = []

    for i, row in enumerate(new_data.to_dict(orient='records')):
        result = {
            "new_data": row,
            "prediction": int(predictions[i])
            }
        if probabilities is not None:
            result["probabilities"] = list(probabilities[i])
        results.append(result)

    # Save to a JSON file
    with open(output_file, 'w') as file:
        json.dump(results, file, indent=4)

    print(f"Inference results saved to {output_file}")


In [ ]:
model = load_model('CP_LR_Gender.pkl')

# Define feature columns (must match the training data)
feature_columns = ['Daily_Time_Spent_on_Site', 'Age', 'Area_Income', 'Daily_Internet_Usage', 'Clicked_on_Ad']


# Adding new file path in here
new_data = pd.read_csv('advertising_ef.csv')

rawdata = pd.DataFrame(new_data)

clean_data = preprocessing(new_data)
predictions, probabilities = make_prediction(model, clean_data)

save_inference_results('Gender_inference_results.json', clean_data, predictions, probabilities, rawdata)

Model loaded successfully from CP_LR_Gender.pkl
Gender: [1 1 1 ... 0 1 0]
Probabilities: [[0.4822626  0.5177374 ]
 [0.49721064 0.50278936]
 [0.4954478  0.5045522 ]
 ...
 [0.54936946 0.45063054]
 [0.49930553 0.50069447]
 [0.54899398 0.45100602]]
Inference results saved to Gender_inference_results.json
